<a href="https://colab.research.google.com/github/amelliaaas/PraktikumML_282-305/blob/main/TugasPraktikum3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **1. Load Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
%cd /content/drive/MyDrive/Covid_data

/content/drive/MyDrive/Covid_data


In [ ]:
# Define kaggle json path
# Sesuaikan dengan path anda
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/Covid_data"

In [ ]:
!ls

kaggle.json


In [ ]:
!kaggle datasets download -d pranavraikokte/covid19-image-dataset

 97% 153M/158M [00:01<00:00, 104MB/s] 
100% 158M/158M [00:01<00:00, 99.1MB/s]


In [ ]:
!ls

covid19-image-dataset.zip  kaggle.json


In [ ]:
!unzip \*.zip &> /dev/null && rm *.zip

In [ ]:
!ls

Covid19-dataset  kaggle.json


In [ ]:
!mkdir dataset

In [ ]:
import os
base_dir = 'Covid19-dataset'

train_dir = os.path.join(base_dir, 'test')
test_dir = os.path.join(base_dir, 'train')

train_covid_dir = os.path.join(train_dir, 'Covid')
train_normal_dir = os.path.join(train_dir, 'Normal')
train_pneumonia_dir = os.path.join(train_dir, 'Viral Pneumonia')

test_covid_dir = os.path.join(test_dir, 'Covid')
test_normal_dir = os.path.join(test_dir, 'Normal')
test_pneumonia_dir = os.path.join(test_dir, 'Viral Pneumonia')


In [ ]:
ls

Covid19-dataset/  dataset/  kaggle.json


In [ ]:
test_covid_dir

'Covid19-dataset/train/Covid'

In [ ]:
!cd -

/


In [ ]:
ls

Covid19-dataset/  dataset/  kaggle.json


In [ ]:
#moving all dataset to training
import shutil

dataset1= os.listdir(test_covid_dir)
for file in dataset1:
  new_path = shutil.move(f"{test_covid_dir}/{file}",train_covid_dir)

dataset2= os.listdir(test_normal_dir)
for file in dataset2:
  new_path = shutil.move(f"{test_normal_dir}/{file}",train_normal_dir)

dataset3= os.listdir(test_pneumonia_dir)
for file in dataset3:
  new_path = shutil.move(f"{test_pneumonia_dir}/{file}",train_pneumonia_dir)



In [ ]:
!ls dataset

**2. Spliting Data**

In [ ]:
pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio(train_dir, output="Covid19-dataset/datasets", seed=1337, ratio=(.7, .15, .15))

Copying files: 312 files [00:05, 57.36 files/s]


**3. Prerocessing**

In [ ]:
# Definisikan path untuk data train test dan validation
# sesuaikan dengan path anda
import os
base_dir = 'Covid19-dataset/datasets'
train_path = os.path.join(base_dir, 'train')
test_path  = os.path.join(base_dir, 'test')
val_path   = os.path.join(base_dir, 'val')

In [ ]:
# TULIS KODE ANDA DISINI

from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

batch_size = 32
augmen_gen = ImageDataGenerator(rescale=1. / 255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest')

train_gen = augmen_gen.flow_from_directory(
    train_path,
    class_mode = 'categorical',
    shuffle=True,
    target_size = (224,224),
    batch_size=batch_size,
    color_mode ='rgb'
)

val_gen = augmen_gen.flow_from_directory(
    val_path,
    class_mode = 'categorical',
    shuffle=True,
    target_size = (224,224),
    batch_size= batch_size,
    color_mode ='rgb'
)

test_gen = augmen_gen.flow_from_directory(
    test_path,
    class_mode = 'categorical',
    shuffle=False,
    target_size = (224,224),
    batch_size=batch_size,
    color_mode ='rgb',
)

FileNotFoundError: ignored

In [ ]:
# IMPORT SEMUA PAKET YANG ANDA PERLUKAN UNTUK MENDEFINISIKAN MODEL DISINI

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
import time

In [ ]:
# DEFINISIKAN MODEL ANDA DISINI

#Feature Selection
model = Sequential()

model.add(InputLayer(input_shape=[224,224,3]))
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))


In [ ]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 56, 56, 128)       73856     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 28, 28, 128)     

In [ ]:
# TULIS CODE CALLBACKS ANDA DISINI

best_model_path = 'best_model.h5'
checkpoint_callback = ModelCheckpoint(best_model_path,
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)
reduce_callback = ReduceLROnPlateau(monitor='val_accuracy', patience=3, factor=0.5, min_lr=0.000003, verbose=1)
callbacks_list = [checkpoint_callback, reduce_callback]

In [ ]:
# TULIS CODE UNTUK DEFINISI LEARNING RATE DISINI 

learning_rate = 0.00001

In [ ]:
# TULIS CODE UNTUK TRAINING DISINI

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=learning_rate),
              metrics='accuracy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# TULIS CODE UNTUK MENCETAK LAMA TRAINING TIME DISINI

start = time.time()
H = model.fit_generator(train_gen, epochs=100,validation_data=val_gen, verbose=1, callbacks=[callbacks_list])

End = time.time()
print ("training time: ", End-start,"s")  # the time would be round to 3 decimal in seconds

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/100
7/7 [==============================] - ETA: 0s - loss: 0.5854 - accuracy: 0.7870
Epoch 00001: val_accuracy improved from 0.31111 to 0.35556, saving model to best_model.h5
7/7 [==============================] - 44s 6s/step - loss: 0.5854 - accuracy: 0.7870 - val_loss: 1.0467 - val_accuracy: 0.3556 - lr: 3.0000e-06
Epoch 2/100
7/7 [==============================] - ETA: 0s - loss: 0.5846 - accuracy: 0.7176
Epoch 00002: val_accuracy improved from 0.35556 to 0.40000, saving model to best_model.h5
7/7 [==============================] - 44s 6s/step - loss: 0.5846 - accuracy: 0.7176 - val_loss: 1.0476 - val_accuracy: 0.4000 - lr: 3.0000e-06
Epoch 3/100
7/7 [==============================] - ETA: 0s - loss: 0.5635 - accuracy: 0.7407
Epoch 00003: val_accuracy improved from 0.40000 to 0.44444, saving model to best_model.h5
7/7 [==============================] - 43s 6s/step - loss: 0.5635 - accuracy: 0.7407 - val_loss: 1.0334 - val_accuracy: 0.4444 - lr: 3.0000e-06
Epoch 4/100
7/7 [==